In [12]:
#Uncomment the next line and run if plotly and chart-studio modules are not installed
#!pip install plotly chart-studio 

In [13]:
#Imports all necessesary modules to run the model
import numpy as np
import plotly.graph_objs as go
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from IPython.display import display
import math
import chart_studio.plotly as py
import matplotlib.pyplot as plt
import warnings
import McCabeTools as mct

In [14]:
#Opens and reads the antoine coefficient data and passes it into useful variables
#To switch between sets of species, choose which text file to pass below
f = open("long_antoine_coef.txt")
#f = open("antoine_coef.txt")
(antoine_mtrx, antoine_dict,name,a_list,b_list,c_list,t1_list,t2_list,id_num) = mct.readprgm(f)

In [15]:
#Establishes the format of the charts in this notebook
layout=go.Layout(title="Txy")
fig = go.FigureWidget(layout=layout)
fig.layout.titlefont.size = 22
fig.layout.titlefont.family = 'Rockwell'
fig.layout.xaxis.title = 'xy'
fig.layout.yaxis.title = 'T (°C)'
fig.layout.plot_bgcolor = 'rgb(255,255,255)'
fig.layout.xaxis.gridcolor = 'darkgrey'
fig.layout.yaxis.gridcolor = 'darkgrey'

In [5]:
def txy(species1 = name[1] , species2 = name[2] ,p = 1):
    tsat = list()
    step=1000
    with fig.batch_update():
        #Clear traces from the plot
        fig.data = []        
        #Passes Antoine vapor coefficients for selected species
        species1 = antoine_dict[species1]
        species2 = antoine_dict[species2]
        a1 = species1['a']
        b1 = species1['b']
        c1 = species1['c']
        a2 = species2['a']
        b2 = species2['b']
        c2 = species2['c']
        
        #Calculates saturation temperature for given species at given pressure with Antoine equation
        if id_num == 1:
            tsat.append(b1/(a1 - math.log(p,10)) - c1)
            tsat.append(b2/(a2 - math.log(p,10)) - c2)
        else:
            tsat.append(b1/(a1 - math.log(p * 750.062,10)) - c1)
            tsat.append(b2/(a2 - math.log(p * 750.062,10)) - c2)            

        t_space = np.linspace(tsat[0],tsat[1],step)
    
        index_max = tsat.index(max(tsat))
        
        y_list = list()
        x_list = list()
        
        reverse_list = list()
        for element in range(len(t_space)):
            reverse_list.append(t_space[-element-1])
            
        #Calculates saturation pressures of given species at given pressure
        for n in reverse_list:
            if id_num == 1:
                p1sat = 10**(a1- (b1/(c1+n)))
                p2sat = 10**(a2- (b2/(c2+n)))
            else:
                p1sat = (10**(a1- (b1/(c1+n)))) / 750.062
                p2sat = (10**(a2- (b2/(c2+n)))) / 750.062
                
            if index_max == 0:
                x_element = float((p-p2sat)/(p1sat-p2sat))
                y_element = float((x_element*p1sat)/(p))
                
            else:
                x_element = float((p-p1sat)/(p2sat-p1sat))
                y_element = float((x_element*p2sat)/(p))  
                
            if x_element == -0.0:
                x_element = 0.0
                
            if y_element == -0.0:
                y_element = 0.0
                
            x_list.append(x_element)
            y_list.append(y_element)
            
        #Determines more and less volatile species in binary mixture
        if index_max == 1:
            volatile = species1['name']
            less_volatile = species2['name']
        else:
            volatile = species2['name']
            less_volatile = species1['name']
            
        #Creates title for graph
        fig.layout.title = volatile + " - " + less_volatile + ",   P=" + str(p) + " bar"
        
        #Adds x and y data to the graph
        xa = fig.add_scatter(name=volatile, x=x_list, y=t_space)
        ya = fig.add_scatter(name=less_volatile, y=t_space, x=y_list)
        
        #Packages x and y data into lists
        vle = list()
        reverse_x = list()
        reverse_y = list()
        for element in range(len(x_list)):
            reverse_x.append(x_list[-element - 1])
            reverse_y.append(y_list[-element -1])
        vle.append(reverse_x)
        vle.append(reverse_y)
        
    #Extracts x and y data from vle lists into x_op and y_op lists
    x_op = vle[1]
    y_op = vle[0]

    return(fig,vle,volatile,less_volatile,reverse_x,reverse_y,x_op,y_op)

#Runs thermo function txy() to generate a graph
(final_graph,vle,volatile,less_volatile,reverse_x,reverse_y,x_op,y_op) = txy()

In [6]:
#Displays dynamic graph of Txy thermo data
z = interactive(txy, species1=antoine_dict.keys(),
                species2=antoine_dict.keys(), 
                p=widgets.BoundedFloatText(value=1,min=0.01,max =100.0,step=0.00001))               
display(z)
display(final_graph)

interactive(children=(Dropdown(description='species1', index=1, options=('1,1,1-trifluoroethane -3-27°C ', '1,…

FigureWidget({
    'data': [{'name': '1,1,2-trichloroethane 29-155°C ',
              'type': 'scatter',
     …

In [7]:
#Establishes layout for McCabe-Theiele graph
layout1=go.Layout()
fig1 = go.FigureWidget(layout=layout1)
fig1.layout.titlefont.size = 22
fig1.layout.titlefont.family = 'Rockwell'
fig1.layout.xaxis.title = 'x'
fig1.layout.yaxis.title = 'y'
fig1.layout.plot_bgcolor = 'rgb(255,255,255)'
#Toggles Gridlines
fig1.layout.xaxis.gridcolor = 'darkgrey'
fig1.layout.yaxis.gridcolor = 'darkgrey'
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=np.RankWarning)
tr_list = ['Off','On']

def op_line(order,x,y): 
    #Performs a simple polynomial fit on the given x,y thermo data
    z = np.polyfit(x,y,order)
    f = np.poly1d(z)
    return(f)

def rootfinder(eqn,sc):
    #Solves the polynomial trend line to calculate intercepts when creating step line
    sc += 1
    output = np.roots(eqn)
    return(output,sc)

def update(xb = 0.25, xd = 0.6, xf = 0.325, r = 9.5, q = 0.7,species1=name[1],species2=name[2],p=1,order=7,TotalReflux="Off",R_min = "Off",Eff=1): 
    sanitycounter = 0
    sc1 = 0
    #Re-calculates thermo data any time species or pressure are modified
    (final_graph,vle,volatile,less_volatile,reverse_x,reverse_y,x_op,y_op) = txy(species1,species2,p)
    x_input = x_op
    y_input = y_op

    with fig1.batch_update():
        #Re-calculates trend line any time the order or thermo data are modified
        z = op_line(order,x_op,y_op)
        
        #Clear traces from the plot and creates title
        fig1.data = []
        step = 100

        #Create the 45 line and add to chart
        ff = fig1.add_scatter(name='45',x=[0,1],y=[0,1])

        #Add the raw data to the chart
        op = fig1.add_scatter(name='Raw Data',x=x_op,y=y_op)
        
        #Creates a series of points that lie on the trendline and then adds that series to the chart
        trend_x = list()
        trend_y = list()
        x_gen = np.linspace(0,1,step)
        for w in range(1,len(x_gen)):
            x_number = x_gen[w]
            trend_y.append(z(x_number))
            trend_x.append(x_number)
        trendlineseries = fig1.add_scatter(name= 'Trendline',x=trend_x,y=trend_y)
        
        #Converts poly1d equilibrium polynomial to useable text equation
        coef_list = z.coef
        order = z.o
        trend_list = list()
        trend_str = str()
        for coef_number in range(order + 1):
            if coef_number is (order):
                trend_fraction_str = str(coef_list[coef_number])
            else:
                trend_fraction_str = str(coef_list[coef_number]) + "x**" + str(order - coef_number ) + " + "
            trend_str += trend_fraction_str
        
        #Creates data for rectifying, stripping, and q lines (and their intersection)
        rec_line_str = str()
        r_slope = (r / (r + 1))
        r_int = xd * (1 / (r + 1))

        if (q ==1 )is True :
            rec_q_int = xf
        else:
            m = q / (q-1)
            q_int = xf / (q-1)
            rec_q_int = (q_int + r_int) / (m - r_slope)
        rec_line_y = (rec_q_int * r_slope) + r_int
        
        #Adds rectifyinig line to chart
        rec_line = fig1.add_scatter(name = 'Rectifying',x=[xd, rec_q_int],y=[xd, rec_line_y])
        
        #Adds feed line (q) to chart
        q_line = fig1.add_scatter(name = 'q',x=[xf, rec_q_int],y=[xf, rec_line_y])
        
        strip_slope = (xb - rec_line_y) / (xb - rec_q_int)
        strip_int = xb - (strip_slope * xb)
        
        #Adds stripping line to chart
        strip_line = fig1.add_scatter(name = "Stripping",x=[xb, rec_q_int],y=[xb, rec_line_y])
        assert strip_slope > 1, "Operating parameters out of range"
        assert rec_line_y < z(rec_q_int), "Pinch point has been created, distillation will never complete"
               
        #Sets up some variables for the step line
        step_x_list = [xd]
        step_y_list = [xd]
        step_x_rmin_list = [xd]
        step_y_rmin_list = [xd]
        eq_value_y_rmin = xd
        eq_value_x_rmin = xd
        eq_value_y = xd
        eq_value_x = xd
        ff_value = xd
        ff_value_rmin = xd
        
        #Creates line stepping down to xb
        while eq_value_x > xb:
            #Solves the root values of the polynomial curve at the 45 line y value
            z_scaled = z - ff_value
            (roots,sanitycounter) = rootfinder(z_scaled,sanitycounter)
            
            #Asserts that the polynomial can be solved
            assert sanitycounter < 500, "Error in solving polynomial, these two components may not be suitable for this model or may require a more accurate trendline"
            
            old_eq_value_x = eq_value_x
            
            #Finds the non negative solution of the polynomial (x value) for the given 45 line y value
            for zero in roots:
                zero_str = str(zero)
                assertion = not("+0j" in zero_str)
                if assertion == True:
                    zero = -1
                else:
                    zero = float(zero_str[1:-4])
                if 0<zero<1 == True:
                    eq_value_x = zero
                    
            
            #Adds a point to the step y list on the equilibrium line directly across from the 45 (same y)    
            if ff_value == xd :
                eq_value_y = xd 
            else:
                eq_value_y = z(eq_value_x) 
            
            eq_value_x = old_eq_value_x -((old_eq_value_x - eq_value_x) * Eff)
                
            #Adds the next 45 line value based on position relative to the stripping and rectifying lines     
            if (eq_value_x > rec_q_int) == True:
                ff_value = eq_value_x * r_slope + r_int 
            else:
                if (eq_value_x > xb) == True:
                    ff_value = (eq_value_x * strip_slope) + strip_int
                else:
                    ff_value = eq_value_x
            step_x_list.append(eq_value_x)
            step_y_list.append(eq_value_y)
            step_x_list.append(eq_value_x)
            step_y_list.append(ff_value) 

        #Adds operating condition step series to graph
        step_line = fig1.add_scatter(name = 'Step',x=step_x_list,y=step_y_list)
        
        #Calculates N and Nmin values
        nmin_value = int((len(step_x_rmin_list) - 1) / 2)
        n_value = int((len(step_x_list) - 1) / 2)
        
        #Calculates R_min
        if R_min =='On':
            r_x_list = [xb]
            r_y_list = [xb]
            
            if q == 1:
                #Catches the case where the q line has an infinite slope
                r_y_value = (z(xf))
                r_min_final_root = (xf)
            else:
                #Packages
                q_line_poly1d = np.poly1d([m,-q_int])
                z_scaled_again = z - q_line_poly1d
                roots_of_newz = z_scaled_again.r
                for zero in roots_of_newz:
                    zero_str = str(zero)
                    assertion = not("+0j" in zero_str)
                    if assertion == True:
                        zero = -1
                    else:
                        zero = float(zero_str[1:-4])
                    if 0<zero<1 == True:
                        r_min_final_root = zero 
                r_y_value = z(r_min_final_root)
                
            if r_y_value > xd:
                r_min_final_root = step_x_list[1]
                r_y_value = xd
                    
            r_y_list.append(r_y_value)
            r_x_list.append(r_min_final_root)
            r_x_list.append(xd)
            r_y_list.append(xd)
            pinchpt = fig1.add_scatter(name = 'R_min',x=r_x_list,y=r_y_list)
            r_min_slope = (xd - r_y_list[-2])/(xd - r_x_list[-2])
            r_min_final_value = r_min_slope/(1-r_min_slope)
            print('R_min is: ',r_min_final_value)            
            
        if TotalReflux == 'On':
            #Calculates the step lines at total reflux
            while eq_value_x_rmin > xb:        
                #Solves for roots of the trendline
                z_scaled_rmin = z - ff_value_rmin
                (roots_rmin,sc1) = rootfinder(z_scaled_rmin,sc1)

                #Asserts that the polynomial trendline can be solved 
                assert sc1 < 500,"Error in solving polynomial, these two components may not be suitable for this model or may require a more accurate trendline"
                for zero in roots_rmin:
                    zero_str = str(zero)
                    assertion = not("+0j" in zero_str)
                    if assertion == True:
                        zero = -1
                    else:
                        zero = float(zero_str[1:-4])
                    if 0<zero<1 == True:
                        eq_value_x_rmin = zero     

                #Calculates step point on 45 line
                if ff_value_rmin == xd:
                    eq_value_y_rmin = xd
                else:
                    eq_value_y_rmin = z(eq_value_x_rmin)
                ff_value_rmin = eq_value_x_rmin


                #Builds total reflux step data series
                step_x_rmin_list.append(eq_value_x_rmin)
                step_y_rmin_list.append(eq_value_y_rmin)
                step_x_rmin_list.append(eq_value_x_rmin)
                step_y_rmin_list.append(ff_value_rmin)
            nmin = fig1.add_scatter(name = 'Total Reflux',x=step_x_rmin_list,y=step_y_rmin_list)
            nmin_value = int((len(step_x_rmin_list) - 1) / 2)
            fig1.layout.title = "McCabe Thiele  "+volatile+"-"+less_volatile + ",   P=" + str(p) + " bar" + "    N=" + str(n_value) + "    Nmin=" + str(nmin_value)
        else:
            #Creates the title of the graph
            fig1.layout.title = "McCabe Thiele  "+volatile+"-"+less_volatile + ",   P=" + str(p) + " bar" + "    N=" + str(n_value)
    
    return(fig1)

final_graph1 = update()

In [10]:
abcd=interactive(update,species1=antoine_dict.keys(),species2=antoine_dict.keys(),
    r=widgets.BoundedFloatText(value=10,min=0.0,max=100.0,step=0.001),
    xd=widgets.BoundedFloatText(value=0.9, min=0.0,max=1.0,step=0.01),
    xb=widgets.BoundedFloatText(value=0.1, min=0.0,max=1.0,step=0.01),
    q=widgets.BoundedFloatText(value=0.7, min=0.0,max=1.0,step=0.01),
    xf=widgets.BoundedFloatText(value=0.5,min=0.0,max=1.0,step=0.01),
    p=widgets.BoundedFloatText(value=1,min=0.01,max =500.0,step=0.00001),
    order=widgets.BoundedFloatText(value=10,min=1,max=30,step=1),TotalReflux=tr_list, R_min=tr_list,
    Eff=widgets.BoundedFloatText(value=1,min=0.001,max=1,step=0.001))
display(abcd)
#final_graph1

interactive(children=(BoundedFloatText(value=0.1, description='xb', max=1.0, step=0.01), BoundedFloatText(valu…

In [11]:
final_graph1

FigureWidget({
    'data': [{'name': '45', 'type': 'scatter', 'uid': '1b95bd95-5427-4183-84f1-72f9c7cd7c02', '…